In [30]:
import matplotlib.pyplot as plt
import xlsxwriter
import numpy as np
import pandas as pd
import pandas.io.sql
import os
import pdfkit
import pytz 
from datetime import datetime 
import base64
import pymssql
import psycopg2 as pg
from jinja2 import Environment, FileSystemLoader

host        = '192.168.1.13'
database    = 'igu_live'
user        = 'sa'
password    = 'B1admin'


host2        = "192.168.9.235"
database2    = "igportal"
user2        = "invoice"
password2    = "invoice" 

conn2 = pg.connect("host="+ host2 + " dbname="+  database2 + " user="+ user2 + " password="+ password2 + "")
 
conn = pymssql.connect(host=host, user=user, password=password, database=database)

msg_sql= """                
                select    
                        c.cardcode  ,
                        c.cardname 'Partner Name',
                        d.groupname 'Group',
                        c.U_AR_Person , 
                        sum(case 
                                when datediff(day,a.refdate,getdate())<=30 and a.transtype in (13,14) then (a.BalScDeb -a.balsccred ) 
                                else 0
                        end) '0-30',  
                        sum(case 
                                when datediff(day,a.refdate,getdate()) between 31 and 60  and a.transtype in (13,14)  then (a.BalScDeb -a.balsccred ) 
                                else 0
                        end) '31-60',  
                        sum(case 
                                when datediff(day,a.refdate,getdate())  between 61 and 90  and a.transtype in (13,14)  then (a.BalScDeb -a.balsccred ) 
                                else 0
                        end) '61-90',  
                        sum(case 
                                when datediff(day,a.refdate,getdate())  between 91 and 120    and a.transtype in (13,14)  then (a.BalScDeb -a.balsccred ) 
                                else 0
                        end) '91-120',  
                        sum(case 
                                when datediff(day,a.refdate,getdate()) >=121   and a.transtype in (13,14)  then (a.BalScDeb -a.balsccred ) 
                                else 0
                        end) '121+',  
                        
                        sum(a.BalScDeb -a.balsccred ) 'Total'
                from jdt1 a 
                inner join ojdt b on a.transid = b.transid 
                inner join ocrd c on a.ShortName = c.cardcode 
                inner join ocrg d on d.groupcode = c.groupcode 
                where 
                        a.account ='1130001' 
                        and a.BalScDeb -a.balsccred  <>0 
                        and convert(varchar,a.refdate,112)<= convert(varchar,getdate(),112)
                group by  
                        c.cardcode ,
                        d.groupname ,
                        c.cardname ,
                        c.U_AR_Person   
                
        """

pandas.options.display.float_format = '{:,.2f}'.format

#msg_sql = """ exec [dbo].[IGU_AGING_AR] '20200531','IGU' """
data = pandas.io.sql.read_sql(msg_sql,conn)

msg_sql2= """
              select  
                            a.salesgroup as "Sales Group" ,  
                            a.salesperson as "Sales Person",
                            a.groupname as "Customer Group",
                            a.ar_person as "AR Person",
                            A.cardcode cardcode,
                            a.cardname   Customer, 
                            a.lock_limit , 
                            a.paymentGroup as "Term of Payment",
                            a.creditline as "Credit limit",
                            a.b60 as "Before 60days" ,
                            a.a60 as "After 60days",
                            a.balance "Balance",
                            (a.creditline - a.balance) as "Remain Credit" ,
                            a.followup_by ,
                            a.laststatus_date,
                            '[' || coalesce(a.followup_type,'') || '] ' || coalesce(a.laststatus ,'') status
                        from sap_bp  A
                        where a.a60 >0
                        order by 
                            a.salesgroup ,  a.salesperson,
                            a.groupname ,  A.cardcode
        """


data2 = pandas.io.sql.read_sql(msg_sql2,conn2)

 
#data
new_df = pandas.merge(data, data2,  on=["cardcode"])
new_df[["Sales Group","Sales Person","AR Person","cardcode","customer","Customer Group","Term of Payment","Credit limit","0-30","31-60","61-90","91-120","121+","Total","Remain Credit","followup_by","status"]]

new_df.to_excel("followup.xlsx")
#
# new_df[["cardcode"]]
#print(data)
#data3 =pandas.merge()




,Sales Group,Sales Person,AR Person,cardcode,customer,Customer Group,Term of Payment,Credit limit,0-30,31-60,61-90,91-120,121+,Total,Remain Credit,followup_by,status
0,HOREKA,[NI] NIA,NOVITA,CT0037,PT. KITCHENETTE LESTARI,Catering,30 Days,"179,000,000.00","13,379,996.00","1,583,999.00",0.00,0.00,"4,859,994.00","19,823,989.00","159,476,011.00",ar,"[phone] PAID 14/08/20 105,859,830\n"
1,HOREKA,[YL] YULI,MEGA,CT0044,PT. PURANTARA MITRA ANGKASA DUA,Catering,30 Days,"500,000,000.00",0.00,0.00,0.00,0.00,"274,257,758.00","274,257,758.00","225,742,242.00",ar,[mail] Tgl 03/08/20 email menanyakan jdwal pem...
2,HOREKA,[DI] DIAN,FENI,CT0112,JESSE SAMUEL,Catering,30 Days,"10,000,000,000.00",0.00,0.00,0.00,0.00,"1,008,024,593.00","1,008,024,593.00","8,991,975,407.00",ar,[whatsapp] PEMBAYARAN DICICIL TIAP AWAL BULAN ...
3,HOREKA,[FR] FERRY,ANATA,HT0061,THE HERMITAGE HOTEL,Hotel,30 Days,"250,000,000.00",0.00,0.00,0.00,0.00,"135,327,079.00","135,327,079.00","114,672,921.00",ar,[phone] DI TELEPON KE OFFICE TIDAK ADA YANG AN...
4,HOREKA,[FB] FEBBY,RISTY,HT0067,PT. MUTIARA PERMATA MULIA,Hotel,30 Days,"5,000,000,000.00","185,508,575.00","8,100,900.00",0.00,0.00,"951,617,836.00","1,145,227,311.00","3,854,772,689.00",ar,[phone] TELPON LAGI KE IBU CHRISTINE JUGA TETA...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,KARYAWAN,[K1] KARYAWAN,CHRISTY,UM0371,INDOGUNA PASTRY,Karyawan,Transfer (7 days from delivery),"30,000,000.00","124,000.00","347,300.00","1,129,499.00",0.00,0.00,"1,600,799.00","28,109,201.00",ar,[]
596,WET,[FY] FINKY,FENI,UM0473,BPK. CECEP,Umum,Transfer,"30,000,000.00",0.00,0.00,0.00,0.00,"4,760,000.00","4,760,000.00","25,240,000.00",ar,[others] SEDANG PENGAJUAN PENGHAPUSAN
597,GROUP,[IN] INTERNAL,MEGA,UM0552,PT. INDOKULINA SARANA UTAMA,Group Jkt,Transfer,"300,000,000,000.00","1,591,306,816.00","1,869,590,866.00","1,790,137,214.00","1,269,730,300.00","21,584,336,348.00","28,105,101,544.00","271,996,356,864.00",ar,[]
598,HOREKA,[FR] FERRY,MARIA,UM0645,NORI RAMA,Umum,Cash,"30,000,000.00",0.00,0.00,0.00,0.00,"5,347,400.00","5,347,400.00","24,652,600.00",ar,[]
